Author: Jinze Wu

Student Number: p09323028

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

# **Download data**
Download data from google cloud

In [ ]:
!gdown --id '1oTtu7K5NVLCp-RKUuyVtwShLnryIVrKc' --output creditcard.xlsx
!ls

Downloading...
From: https://drive.google.com/uc?id=1oTtu7K5NVLCp-RKUuyVtwShLnryIVrKc
To: /content/creditcard.xlsx
100% 361k/361k [00:00<00:00, 47.4MB/s]
creditcard.xlsx  sample_data


# **Sheet to DataFrame**

In [ ]:
customer = pd.read_excel('creditcard.xlsx', sheet_name='客戶資料檔')
creditcard = pd.read_excel('creditcard.xlsx', sheet_name='信用卡資料檔')
creditlog = pd.read_excel('creditcard.xlsx', sheet_name='信用卡交易記錄檔')
creditlog = creditlog.iloc[:,:8]

In [ ]:
customer.head()

,客戶ID,年齡,生日,居住地,教育程度,性別,婚姻狀況,職業,辦第一張信用卡的時間
0,7293,57,1955-08-24,大台北地區,專科,1:男,已婚,公務員,2006-11-27
1,5963,29,1983-12-05,大台北地區,學士,1:男,單身,白領一般職員,2009-07-24
2,18700,27,1985-12-20,高屏地區,高中,1:男,單身,藍領(勞動工作或作業員)一般職員,2009-09-22
3,920,39,1973-05-29,大台北地區,學士,1:男,已婚,自營事業老闆,2010-03-05
4,4612,26,1986-01-06,嘉南地區,學士,1:男,單身,白領一般職員,2010-08-05


In [ ]:
creditcard.head()

,信用卡ID,客戶ID,信用卡開卡日,信用卡到期日,信用額度,卡等
0,41,2027,2006-05-19,2015-08-31,500000,金卡
1,814,7437,2002-01-14,2016-01-31,200000,普卡
2,905,6475,2002-05-07,2016-05-31,50000,普卡
3,1142,18841,2001-07-23,2015-07-31,250000,金卡
4,1411,14127,2005-10-03,2016-06-30,270000,白金卡


In [ ]:
creditlog.head()

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
0,8,6448,15195,2010-12-01,14_旅遊,一般消費,5200,國內
1,9,6826,13687,2010-12-01,12_量販超市,一般消費,250,國內
2,10,6826,13687,2010-12-01,12_量販超市,一般消費,1316,國內
3,11,7896,179,2010-12-01,10_藥妝,一般消費,218,國內
4,20,9387,16756,2010-12-01,11_3C居家電信,一般消費,2610,國內


# **Intervalpurchase Time**

In [ ]:
df = creditlog.copy()
df

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
0,8,6448,15195,2010-12-01,14_旅遊,一般消費,5200,國內
1,9,6826,13687,2010-12-01,12_量販超市,一般消費,250,國內
2,10,6826,13687,2010-12-01,12_量販超市,一般消費,1316,國內
3,11,7896,179,2010-12-01,10_藥妝,一般消費,218,國內
4,20,9387,16756,2010-12-01,11_3C居家電信,一般消費,2610,國內
...,...,...,...,...,...,...,...,...
7759,30520,12797,19483,2012-11-30,08_精品,一般消費,3760,國內
7760,30529,17200,15353,2012-11-30,12_量販超市,一般消費,262,國內
7761,30533,17746,17586,2012-11-30,16_無店舖,一般消費,1040,國內
7762,30535,18122,3219,2012-11-30,13_交通(含加值),一般消費,197,國內


In [ ]:
df.drop_duplicates(['客戶ID', '刷卡日期'], inplace=True)

In [ ]:
df

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
0,8,6448,15195,2010-12-01,14_旅遊,一般消費,5200,國內
1,9,6826,13687,2010-12-01,12_量販超市,一般消費,250,國內
3,11,7896,179,2010-12-01,10_藥妝,一般消費,218,國內
4,20,9387,16756,2010-12-01,11_3C居家電信,一般消費,2610,國內
5,36,12989,3044,2010-12-01,06_公用事業,一般消費,20,國內
...,...,...,...,...,...,...,...,...
7759,30520,12797,19483,2012-11-30,08_精品,一般消費,3760,國內
7760,30529,17200,15353,2012-11-30,12_量販超市,一般消費,262,國內
7761,30533,17746,17586,2012-11-30,16_無店舖,一般消費,1040,國內
7762,30535,18122,3219,2012-11-30,13_交通(含加值),一般消費,197,國內


In [ ]:
df = df.sort_values(['客戶ID', '刷卡日期'])

In [ ]:
df['shift'] = (df['客戶ID'] == df['客戶ID'].shift(1))
df['IPT'] = (df['刷卡日期'] - df['刷卡日期'].shift(1))

In [ ]:
df.drop(df[df['shift'] == False].index, inplace=True)
df

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點,shift,IPT
263,1016,8905,89,2010-12-24,13_交通(含加值),一般消費,1623,國內,True,10 days
414,1546,8905,89,2011-01-05,13_交通(含加值),一般消費,1545,國內,True,12 days
510,2011,8905,89,2011-01-17,13_交通(含加值),一般消費,1550,國內,True,12 days
637,2525,14554,89,2011-01-26,12_量販超市,一般消費,893,國內,True,9 days
726,2850,8905,89,2011-02-08,07_百貨,一般消費,154,國內,True,13 days
...,...,...,...,...,...,...,...,...,...,...
7334,28976,6619,19859,2012-10-25,09_餐飲,一般消費,726,國內,True,8 days
7395,29214,14318,19859,2012-10-30,07_百貨,一般消費,300,國內,True,5 days
7458,29470,6619,19859,2012-11-06,09_餐飲,一般消費,270,國內,True,7 days
7549,29805,6619,19859,2012-11-14,16_無店舖,一般消費,1573,國內,True,8 days


In [ ]:
CAI = df.drop_duplicates(subset=['客戶ID'], ignore_index=True)[['客戶ID']]

In [ ]:
CAI['刷卡次數'] = CAI['客戶ID'].map(creditlog['客戶ID'].value_counts())
CAI['消費日數'] = CAI['客戶ID'].map(df['客戶ID'].value_counts()) + 1
CAI['MLE'] = CAI['客戶ID'].map(df.groupby(['客戶ID'])['IPT'].apply(lambda x: np.mean(x).days))
# CAI['MLE'] = CAI['客戶ID'].map(df.groupby(['客戶ID'])['IPT'].mean(numeric_only=False))
CAI

,客戶ID,刷卡次數,消費日數,MLE
0,89,85,69,10
1,106,75,67,10
2,131,16,15,23
3,179,4,4,147
4,261,13,12,32
...,...,...,...,...
94,19239,13,12,59
95,19483,107,91,7
96,19504,98,64,11
97,19639,201,162,4
